## Prepare maintenance documents

In [0]:
manual_bearings = """Title: Motor Bearing Inspection and Replacement Guide

1. Purpose:
   Bearings should be inspected when vibration exceeds 3.5 mm/s or temperature rises above 55°C.

2. Symptoms of bearing wear:
   - Increased vibration or noise
   - Abnormal heat near shaft
   - Reduced RPMs or power draw increase

3. Recommended Actions:
   - Shut down equipment safely
   - Use IR thermometer to confirm localized heating
   - Check for play in shaft coupling
   - Replace both drive-end and non-drive-end bearings
   - Log bearing condition in CMMS after replacement"""


manual_overheating = """Title: Troubleshooting Overheating in Electric Motors

1. Common Causes:
   - Blocked ventilation or dirty cooling fans
   - Electrical overload
   - Shaft misalignment
   - Lubrication failure (dry bearing)

2. Indicators:
   - Temperature exceeding 60°C during normal load
   - Motor casing warm to touch or smelling of burnt insulation
   - Vibrations > 3.2 mm/s

3. Recommended Actions:
   - Clean cooling ducts and check fan blades
   - Verify motor current vs rated amps
   - Inspect for shaft alignment using dial indicators
   - Relubricate bearings or check for grease breakdown"""


manual_presure_drop = """Title: Pressure Drop in Hydraulic Systems

1. Normal operating pressure: 30-35 PSI
2. Warning level: < 28 PSI
3. Possible Causes:
   - Internal leak in actuator or valve
   - Blockage in fluid line
   - Air entrapment or low fluid level

4. Fix Procedures:
   - Check reservoir level and refill if low
   - Bleed air from the system using purge valves
   - Inspect filters and lines for clogs or wear
   - Run system diagnostics to isolate fault location"""


manual_lubrication = """Title: Lubrication Maintenance Procedures for Rotary Equipment

1. Purpose:
   Ensure adequate lubrication to prevent mechanical wear and overheating.

2. Recommended Lubrication Intervals:
   - Bearings: every 500 operational hours
   - Gearboxes: every 1,000 operational hours or per OEM spec
   - Use high-grade synthetic grease (NLGI 2) for high-speed shafts

3. Symptoms of Lubrication Failure:
   - Rising temperature (> 55°C) near bearing housing
   - Increased noise or vibration (> 3.2 mm/s)
   - Visible oil leakage or grease seal failure

4. Troubleshooting and Repair:
   - Inspect seal condition and replace if faulty
   - Clean old grease and apply fresh lubricant uniformly
   - Monitor vibration for stabilization after lubrication
   - Record in CMMS with grease type and batch number"""


manual_alignment = """Title: Shaft Alignment and Coupling Alignment Guide

1. Purpose:
   Prevent undue stress, vibration, and premature failure due to misalignment.

2. Types of Misalignment:
   - Angular: shaft angles misaligned (> 0.1°)
   - Parallel/Offset: centerline displacement (> 0.1 mm)

3. Symptoms:
   - Persistent vibration (> 3.5 mm/s)
   - Abnormal coupling wear or noise
   - Bearing temperatures elevated by > 5°C above baseline

4. Alignment Procedure:
   - Use dial indicators or laser alignment tools
   - Allow machine to cool to idle temperature before measuring
   - Adjust motor/drive position until angular + parallel errors < 0.05 mm
   - Tighten coupling bolts to OEM torque specs"""


manual_electical_inspection = """Title: Electrical Inspection Procedures for AC Motors

1. Visual Inspection:
   - Check stator windings for discoloration or insulation damage
   - Ensure proper grounding and tight terminal connections

2. Electrical Testing:
   - Use megohmmeter to test insulation resistance (> 1 MΩ)
   - Measure line-to-line and line-to-neutral voltages; allow < 5% imbalance

3. Symptoms of Electrical Fault:
   - Excess heat (> 60°C) at casing or terminal box
   - Erratic motor current, voltage spikes, or tripping
   - Burning smell or smoke traces

4. Recommended Actions:
   - De-energize and allow cooling before opening terminal box
   - Replace damaged windings or insulation immediately
   - Balance power supply and check drive parameters
   - Document meter readings and corrective action taken"""

In [0]:
%pip install databricks-vectorsearch databricks-langchain langchain-community databricks-sdk
dbutils.library.restartPython()


## Load and Chunk the Documents

In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from databricks.vector_search.client import VectorSearchClient
import uuid

In [0]:
docs = [manual_bearings, manual_overheating, manual_presure_drop]
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)

chunks = []
for doc in docs:
  chunks.extend(splitter.split_text(doc))

records = [(str(uuid.uuid4()), chunk) for chunk in chunks]
df_records = spark.createDataFrame(records, schema="id STRING, chunk_text STRING")

In [0]:
# Define Unity Catalog table name
catalog = "workspace"
schema = "genai_demo"
table_name = "maintenance_docs_chunks"

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")

# Save to Delta table
df_records.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.{table_name}")

## Create a Delta Sync Vector Index

### Create a Vector Search Endpoint

In [0]:
from databricks.vector_search.client import VectorSearchClient

client = VectorSearchClient()
endpoint_name = "maintenance_endpoint"

# Create endpoint (Standard or Storage Optimized)
client.create_endpoint(name=endpoint_name, endpoint_type="STANDARD")
client.wait_for_endpoint(name=endpoint_name, verbose=True)
print(f"✅ Endpoint '{endpoint_name}' is ready.")


### Create a Vector Index

In [0]:
spark.sql("""
ALTER TABLE workspace.genai_demo.maintenance_docs_chunks
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

In [0]:
client = VectorSearchClient()

source_table_name = "workspace.genai_demo.maintenance_docs_chunks"
index_name = "workspace.genai_demo.maintenance_docs_index"

index = client.create_delta_sync_index(
  endpoint_name=endpoint_name,
  source_table_name=source_table_name,
  index_name=index_name,
  pipeline_type="TRIGGERED",
  primary_key="id",
  embedding_source_column="chunk_text",
  embedding_model_endpoint_name="databricks-gte-large-en",
)

In [0]:
index.sync()

### Query the index

In [0]:
%pip install databricks-sdk[openai]
dbutils.library.restartPython()

In [0]:
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient

## Query the index and Anomaly root cause analysis

In [0]:
index_name = "workspace.genai_demo.maintenance_docs_index"

ws = WorkspaceClient()
client = VectorSearchClient()
index = client.get_index(index_name=index_name)

# Example anomaly description
anomaly_text = (
  "Machine 3 shows temperature 62°C, vibration 3.9 mm/s, pressure 27 PSI"
)

# Retrieve relevant chunks
res = index.similarity_search(query_text=anomaly_text, columns=["chunk_text"], num_results=1)
context = "\n\n".join([r[0] for r in res["result"]["data_array"]])

# Call LLaMA-4 Maverick via AI Functions
response = ws.serving_endpoints.get_open_ai_client().chat.completions.create(
    model="databricks-llama-4-maverick",
    messages=[
        {"role": "system", "content": "You are an expert maintenance engineer."},
        {"role": "user", "content":
            f"Anomaly:\n{anomaly_text}\n\nContext from manuals:\n{context}\n\n"
            "What is the likely root cause and recommended action?"}
    ]
)

print(response.choices[0].message.content)

## Analyse the root cause of the anomalies in the test dataset

In [0]:
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient

index_name = "workspace.genai_demo.maintenance_docs_index"
ws = WorkspaceClient()
client = VectorSearchClient()
index = client.get_index(index_name=index_name)

def anomlay_root_cause_analysis(sensor_data):
    anomaly_flag = sensor_data['anomaly']
    if anomaly_flag == 1:
        return ''
    
    machine_id = sensor_data['machine_id']
    temperature = sensor_data['temperature']
    vibration = sensor_data['vibration']
    pressure = sensor_data['pressure']
    anomaly_text = (f"Machine {machine_id} shows temperature {temperature}°C, vibration {vibration} mm/s, pressure {pressure} PSI")

    # Retrieve relevant chunks
    res = index.similarity_search(query_text=anomaly_text, columns=["chunk_text"], num_results=1)
    context = "\n\n".join([r[0] for r in res["result"]["data_array"]])

    response = ws.serving_endpoints.get_open_ai_client().chat.completions.create(
        model="databricks-llama-4-maverick",
        messages=[
            {"role": "system", "content": "You are an expert maintenance engineer."},
            {"role": "user", "content":
                f"Anomaly:\n{anomaly_text}\n\nContext from manuals:\n{context}\n\n"
                "What is the likely root cause and recommended action?"}
        ]
    )
    return response.choices[0].message.content


### Generate suggestion for detected machine anomalies

In [0]:
df_test = spark.table("sensor_data_test_scored").toPandas()
df_test['suggestion'] = df_test.apply(anomlay_root_cause_analysis, axis=1)

In [0]:
spark.createDataFrame(df_test).write.mode("overwrite").format("delta").saveAsTable("sensor_data_with_suggestions")

In [0]:
display(df_test.head(100))